In [64]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
import matplotlib.pyplot as plt 
warnings.filterwarnings('ignore')
import gensim
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Bidirectional
from keras.models import Model

<h1 align=center><font size = 5 color='blue'>Chargement et préparation des données</font></h1>

In [65]:
data=pd.read_csv('../data/PubMed-multi-label-dataset.csv')
data.head(10)

,Title,abstractText,meshMajor,pmid,meshid,meshroot,A,B,C,D,E,F,G,H,I,J,L,M,N,Z
0,Expression of p53 and coexistence of HPV in pr...,Fifty-four paraffin embedded tissue sections f...,"['DNA Probes, HPV', 'DNA, Viral', 'Female', 'H...",8549602,"[['D13.444.600.223.555', 'D27.505.259.750.600....","['Chemicals and Drugs [D]', 'Organisms [B]', '...",0,1,1,1,1,0,0,1,0,0,0,0,0,0
1,Vitamin D status in pregnant Indian women acro...,The present cross-sectional study was conducte...,"['Adult', 'Alkaline Phosphatase', 'Breast Feed...",21736816,"[['M01.060.116'], ['D08.811.277.352.650.035'],...","['Named Groups [M]', 'Chemicals and Drugs [D]'...",0,1,1,1,1,1,1,0,1,1,0,1,1,1
2,[Identification of a functionally important di...,The occurrence of individual amino acids and d...,"['Amino Acid Sequence', 'Analgesics, Opioid', ...",19060934,"[['G02.111.570.060', 'L01.453.245.667.060'], [...","['Phenomena and Processes [G]', 'Information S...",1,1,0,1,1,0,1,0,0,0,1,0,0,0
3,Multilayer capsules: a promising microencapsul...,"In 1980, Lim and Sun introduced a microcapsule...","['Acrylic Resins', 'Alginates', 'Animals', 'Bi...",11426874,"[['D05.750.716.822.111', 'D25.720.716.822.111'...","['Chemicals and Drugs [D]', 'Technology, Indus...",1,1,1,1,1,0,1,0,0,1,0,0,0,0
4,"Nanohydrogel with N,N'-bis(acryloyl)cystine cr...",Substantially improved hydrogel particles base...,"['Antineoplastic Agents', 'Cell Proliferation'...",28323099,"[['D27.505.954.248'], ['G04.161.750', 'G07.345...","['Chemicals and Drugs [D]', 'Phenomena and Pro...",1,1,0,1,1,0,1,0,0,1,0,0,0,0
5,"A new <i>Panolis</i> H?bner, [1821] species fr...",Panolis is a well-defined and compact Palearct...,"['Animal Distribution', 'Animals', 'Asia', 'La...",28609947,"[['F01.145.113.069', 'G16.049'], ['B01.050'], ...","['Psychiatry and Psychology [F]', 'Phenomena a...",0,1,0,0,0,1,1,0,0,0,0,0,0,1
6,Characterization of neutron field in a NPP wor...,"At the Krsko Nuclear Power Plant (NPP), albedo...","['Algorithms', 'Equipment Design', 'Equipment ...",17416593,"[['G17.035', 'L01.224.050'], ['E05.320'], ['E0...","['Phenomena and Processes [G]', 'Information S...",0,0,0,0,1,0,1,0,1,1,1,0,1,1
7,Tramadol vs dexmedetomidine for emergence agit...,BACKGROUND: This study was designed to compare...,"['Adenoidectomy', 'Airway Extubation', 'Analge...",28283018,"[['E04.580.068'], ['E02.041.249', 'E05.008'], ...","['Analytical, Diagnostic and Therapeutic Techn...",0,1,1,1,1,1,0,0,0,0,0,1,1,0
8,Nucleotide diphosphates activate the ATP-sensi...,Patch-clamp techniques were used to study the ...,"['Adenosine Diphosphate', 'Adenosine Triphosph...",1488275,"[['D03.633.100.759.646.138.124', 'D13.695.667....","['Chemicals and Drugs [D]', 'Organisms [B]', '...",1,1,0,1,0,0,0,1,0,0,0,0,0,0
9,Using Standardized Checklists Increase the Com...,INTRODUCTION: Hospital evacuations of patients...,"['Checklist', 'Civil Defense', 'Emergencies', ...",31389323,"[['N05.715.360.300.179'], ['I01.451.227'], ['C...","['Health Care [N]', 'Anthropology, Education, ...",0,1,1,0,0,0,1,0,1,0,0,0,1,1


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         49998 non-null  object
 1   abstractText  50000 non-null  object
 2   meshMajor     50000 non-null  object
 3   pmid          50000 non-null  int64 
 4   meshid        50000 non-null  object
 5   meshroot      50000 non-null  object
 6   A             50000 non-null  int64 
 7   B             50000 non-null  int64 
 8   C             50000 non-null  int64 
 9   D             50000 non-null  int64 
 10  E             50000 non-null  int64 
 11  F             50000 non-null  int64 
 12  G             50000 non-null  int64 
 13  H             50000 non-null  int64 
 14  I             50000 non-null  int64 
 15  J             50000 non-null  int64 
 16  L             50000 non-null  int64 
 17  M             50000 non-null  int64 
 18  N             50000 non-null  int64 
 19  Z   

In [67]:
corpus=data.iloc[:,2].values

In [68]:
Y = data.iloc[:, 6:20].to_numpy(dtype=int)
print(Y.shape)  # MUST be (50000, 14)


(50000, 14)


In [69]:
print('Taille du corpus : {0:d}'.format(len(corpus)))

Taille du corpus : 50000


### Ici vous pouvez traiter de votre corpus si vous le souhaitez

In [8]:
#entrainement du model Word2vec sur la colonnes abstracts 

<h1 align=center><font size = 5 color='blue'>Découpage de la base en Apprentissage et Test</font></h1>

In [70]:
corpus_train,corpus_test,y_train,y_test=train_test_split(corpus,Y,test_size=0.33,shuffle=True,random_state=1)

In [71]:
# Tokenization
corpus_train_tokens = [
    gensim.utils.simple_preprocess(text) for text in corpus_train
]

corpus_test_tokens = [
    gensim.utils.simple_preprocess(text) for text in corpus_test
]

In [72]:
print("Exemple du 1er texte du Train :", corpus_train_tokens[0])


Exemple du 1er texte du Train : ['animals', 'cytokines', 'endometrium', 'escherichia', 'coli', 'female', 'goats', 'immunity', 'innate', 'neutrophils', 'postpartum', 'period', 'rna', 'messenger', 'toll', 'like', 'receptor', 'up', 'regulation', 'uterus', 'beta', 'defensins']


<h1 align=center><font size = 5 color='blue'>Développement d'un LSTM utilisant le Word2Vec entraine </font></h1>

In [76]:
model_wv_entraine=gensim.models.Word2Vec.load('modele/word2vec_entraine.h5')
model=model_wv_entraine.wv

In [77]:
print("Taille du vecteur d'embedding du modèle Google Word2Vec :", model.vector_size)
print("Taille du vocabulaire du modèle Word2Vec :", len(model.index_to_key))

Taille du vecteur d'embedding du modèle Google Word2Vec : 200
Taille du vocabulaire du modèle Word2Vec : 12416


<h1 align=left><font size = 4 color='Green'>Préparation des textes pour le modèle LSTM</font></h1>

In [78]:
# Cette fonction Convertit un texte (liste de mots) en séquence d'indices selon le dictionnaire fourni. 
# Seuls les mots présents dans le dictionnaire sont conservés
def to_sequence(index, text):
    indexes = [index[word] for word in text if word in index]
    return indexes

word2idx = {word: idx for idx, word in enumerate(model.index_to_key)}
X_train_sequences = [to_sequence(word2idx, x) for x in corpus_train_tokens]
X_test_sequences = [to_sequence(word2idx, x) for x in corpus_test_tokens]

print("Exemple du 1er texte du train :", corpus_train_tokens[0])
print("1er texte du train converti en indices :", X_train_sequences[0])

Exemple du 1er texte du train : ['animals', 'cytokines', 'endometrium', 'escherichia', 'coli', 'female', 'goats', 'immunity', 'innate', 'neutrophils', 'postpartum', 'period', 'rna', 'messenger', 'toll', 'like', 'receptor', 'up', 'regulation', 'uterus', 'beta', 'defensins']
1er texte du train converti en indices : [5, 480, 2099, 184, 179, 3, 2118, 656, 1667, 711, 2238, 613, 37, 147, 1301, 450, 122, 62, 51, 1357, 119, 4575]


In [79]:
print("Longueur du texte du Train le plus long : ",max([len(X_train_sequences[i]) for i in range(len(X_train_sequences))]),"tokens")

Longueur du texte du Train le plus long :  106 tokens


In [80]:
MAX_SEQ_LENGHT=50 # Longueur maximale des séquences (nombre de tokens conservés)
N_FEATURES = len(model.index_to_key)
# Padding ou troncature des séquences à une longueur fixe de 50 tokens.
# Si la séquence est plus longue : on garde les 50 derniers tokens.
# Si elle est plus courte : on ajoute du padding avec la valeur "N_FEATURES".
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)
print("1er texte du train, indexé et ramené à 50 tokens (en gardant les derniers) :\n", X_train_sequences[0])

1er texte du train, indexé et ramené à 50 tokens (en gardant les derniers) :
 [12416 12416 12416 12416 12416 12416 12416 12416 12416 12416 12416 12416
 12416 12416 12416 12416 12416 12416 12416 12416 12416 12416 12416 12416
 12416 12416 12416 12416     5   480  2099   184   179     3  2118   656
  1667   711  2238   613    37   147  1301   450   122    62    51  1357
   119  4575]


<h1 align=left><font size = 4 color='Green'>Création de la matrice d'embedding</font></h1>

In [81]:
# Taille des vecteurs d'embedding produits par le modèle Google Word2Vec
EMBEDDINGS_LEN = model.vector_size
# Création d'une matrice d'embeddings initialisée à zéro
# Chaque ligne correspond à un mot du vocabulaire, et chaque colonne à une dimension de l'embedding
# On ajoute une ligne supplémentaire à la fin pour le padding (index hors vocabulaire)

embeddings_index = np.zeros((len(model.index_to_key)+1, EMBEDDINGS_LEN))
# Remplissage de la matrice avec les vecteurs du modèle Word2Vec
# La dernière ligne reste à zéro pour représenter les tokens de padding

embeddings_index[:-1,:]=model.vectors
print("Taille de la matrice d'embedding (vocab_size + 1, embedding_dim) :", embeddings_index.shape)


Taille de la matrice d'embedding (vocab_size + 1, embedding_dim) : (12417, 200)


<h1 align=left><font size = 4 color='Green'>Création de l'architecture du modèle LSTM</font></h1>

In [82]:
model_lstm = Sequential()

model_lstm.add(Embedding(
    input_dim=len(model.key_to_index) + 1,
    output_dim=EMBEDDINGS_LEN,
    weights=[embeddings_index],
    trainable=True
))

model_lstm.add(LSTM(100, dropout=0.2))

# ✅ 14 outputs (one per label)
model_lstm.add(Dense(14, activation='sigmoid'))

model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']  # OK for training only
)

model_lstm.summary()



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │     2,483,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,483,400 (9.47 MB)

 Trainable params: 2,483,400 (9.47 MB)

 Non-trainable params: 0 (0.00 B)

<h1 align=left><font size = 4 color='Green'>Entraînement du modèle LSTM</font></h1>

In [83]:
history = model_lstm.fit(X_train_sequences, y_train, epochs=10, batch_size=128, verbose=1,validation_split=0.1)

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 33s 132ms/step - accuracy: 0.3099 - loss: 0.3534 - val_accuracy: 0.3027 - val_loss: 0.2631
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 29s 125ms/step - accuracy: 0.2612 - loss: 0.2474 - val_accuracy: 0.2266 - val_loss: 0.2218
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 28s 120ms/step - accuracy: 0.2074 - loss: 0.2124 - val_accuracy: 0.2260 - val_loss: 0.1969
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 30s 125ms/step - accuracy: 0.1885 - loss: 0.1900 - val_accuracy: 0.1725 - val_loss: 0.1811
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 30s 125ms/step - accuracy: 0.1586 - loss: 0.1732 - val_accuracy: 0.1615 - val_loss: 0.1697
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 29s 124ms/step - accuracy: 0.1626 - loss: 0.1596 - val_accuracy: 0.1493 - val_loss: 0.1576
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 29s 124ms/step - accuracy: 0.1482 - loss: 0.1474 - val_accuracy: 0.1284 - val_loss: 0.1505
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 29s 124ms/step - accuracy: 0.1456 - loss: 0

In [84]:
y_pred_proba = model_lstm.predict(X_test_sequences)
y_pred = (y_pred_proba >= 0.5).astype(int)

516/516 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step


In [85]:
from sklearn.metrics import f1_score

f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1 Macro sur le jeu de test :", f1_macro)


F1 Macro sur le jeu de test : 0.8956500197018678


In [86]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.9172    0.8996    0.9083      7680
           1     0.9858    0.9926    0.9892     15364
           2     0.9549    0.9084    0.9310      8710
           3     0.9704    0.9478    0.9590     10409
           4     0.9631    0.9362    0.9495     12877
           5     0.9431    0.8777    0.9092      2910
           6     0.9425    0.9319    0.9372     11159
           7     0.8367    0.5291    0.6483      1975
           8     0.8637    0.6918    0.7682      1804
           9     0.8819    0.6937    0.7765      1851
          10     0.9682    0.8671    0.9149      2461
          11     0.9838    0.9835    0.9837      6917
          12     0.9684    0.9559    0.9621      7468
          13     0.9436    0.8639    0.9020      2652

   micro avg     0.9571    0.9220    0.9392     94237
   macro avg     0.9374    0.8628    0.8957     94237
weighted avg     0.9554    0.9220    0.9373     94237
 samples avg     0.9573   

In [87]:
for t in [0.3, 0.4, 0.5, 0.6]:
    y_pred = (y_pred_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"Threshold {t}: F1-macro = {f1:.4f}")


Threshold 0.3: F1-macro = 0.9006
Threshold 0.4: F1-macro = 0.9006
Threshold 0.5: F1-macro = 0.8957
Threshold 0.6: F1-macro = 0.8864


# Checking for overfitting of the lstm model 

In [94]:
y_train_pred = (model_lstm.predict(X_train_sequences) >= 0.5).astype(int)
train_f1 = f1_score(y_train, y_train_pred, average='macro')

print("Train Macro F1:", train_f1)
print("Test  Macro F1:", f1_macro)


1047/1047 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step
Train Macro F1: 0.9285021908119307
Test  Macro F1: 0.8956500197018678


The model achieves a Macro F1-score of 0.93 on the training set and 0.90 on the test set. The small performance gap indicates good generalization and no significant overfitting, despite the relatively large number of parameters induced by the embedding layer

<h1 align=left><font size = 4 color='Green'>Évaluation du modèle LSTM sur les données de test</font></h1>

<h1 align=center><font size = 5 color='blue'>Bonus</font></h1>

<h1 align=left><font size = 4 color='Green'>Utilisation de la sortie des LSTM dans d'autres modèles ML</font></h1>

In [88]:
# Affichage des différentes couches du modèle LSTM
print(model_lstm.layers)

# Utilisation d'un modèle dont la sortie correspond à la couche LSTM après l'Embedding
Model_embedding_texts_LSTM = Model(inputs=model_lstm.inputs, outputs=model_lstm.layers[1].output)

# Sauvegarde du modèle intermédiaire pour générer des features
Model_embedding_texts_LSTM.save('../models/model_lstm_features.h5')

# Génération des représentations vectorielles (features) des textes d'entraînement à utiliser pour un fit de votre modèle ML 
corpus_train_lstm = Model_embedding_texts_LSTM.predict(X_train_sequences)

# Génération des représentations vectorielles des textes de test à utiliser pour un predict avec votre modèle ML
corpus_test_lstm = Model_embedding_texts_LSTM.predict(X_test_sequences)

# Affichage de la forme des features générées pour le train
print(corpus_train_lstm.shape)


[<Embedding name=embedding_5, built=True>, <LSTM name=lstm_5, built=True>, <Dense name=dense_4, built=True>]
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step
516/516 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step
(33500, 100)


# Bonus : BI-LSTM

<h1 align=left><font size = 4 color='Green'>Création de larchitecture du modèle BI-LSTM</font></h1>

In [55]:
model_bi= Sequential([
    Embedding(
        input_dim=len(model.key_to_index) + 1,
        output_dim=EMBEDDINGS_LEN,
        weights=[embeddings_index],
        trainable=True
    ),

    Bidirectional(
        LSTM(100, recurrent_dropout=0.2),
        merge_mode='concat'
    ),


    Dense(14, activation='sigmoid')  # multi-label
])


In [89]:
model_bi.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']  # OK for training only
)

model_bi.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 50, 200)        │     2,483,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 200)            │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │         2,814 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,727,014 (10.40 MB)

 Trainable params: 2,727,014 (10.40 MB)

 Non-trainable params: 0 (0.00 B)

In [90]:
history_bi= model_bi.fit(X_train_sequences, y_train, epochs=10, batch_size=128, verbose=1,validation_split=0.1)

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 34s 120ms/step - accuracy: 0.1826 - loss: 0.0468 - val_accuracy: 0.2051 - val_loss: 0.1509
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 28s 117ms/step - accuracy: 0.1901 - loss: 0.0379 - val_accuracy: 0.1755 - val_loss: 0.1545
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 28s 117ms/step - accuracy: 0.1898 - loss: 0.0314 - val_accuracy: 0.1833 - val_loss: 0.1566
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 36s 151ms/step - accuracy: 0.1951 - loss: 0.0269 - val_accuracy: 0.2012 - val_loss: 0.1612
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 37s 158ms/step - accuracy: 0.2000 - loss: 0.0226 - val_accuracy: 0.2042 - val_loss: 0.1635
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 39s 166ms/step - accuracy: 0.2048 - loss: 0.0186 - val_accuracy: 0.2155 - val_loss: 0.1677
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 38s 162ms/step - accuracy: 0.2132 - loss: 0.0165 - val_accuracy: 0.2313 - val_loss: 0.1730
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 36s 151ms/step - accuracy: 0.2189 - loss: 0

In [59]:
y_pred_proba_bi = model_bi.predict(X_test_sequences)
y_pred = (y_pred_proba_bi >= 0.5).astype(int)

516/516 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step


In [60]:
from sklearn.metrics import f1_score

f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1 Macro sur le jeu de test :", f1_macro)


F1 Macro sur le jeu de test : 0.8915017917142963


In [61]:
for t in [0.3, 0.4, 0.5, 0.6]:
    y_pred = (y_pred_proba_bi >= t).astype(int)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"Threshold {t}: F1-macro = {f1:.4f}")


Threshold 0.3: F1-macro = 0.8951
Threshold 0.4: F1-macro = 0.8948
Threshold 0.5: F1-macro = 0.8915
Threshold 0.6: F1-macro = 0.8858
